In [11]:
from itertools import combinations

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import combinations

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import warnings

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from unidecode import unidecode
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from feature_engine import categorical_encoders as ce

from itertools import combinations
from datetime import datetime
from contextlib import contextmanager

In [13]:
NUM_BOOST_ROUND= 10000

DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 34, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]

In [49]:
def subtract_date(date1,date2, df):
    df[date1] = pd.to_datetime(df[date1], infer_datetime_format=True)
    df[date2] = pd.to_datetime(df[date2], infer_datetime_format=True)
    df[date1+date2] = (df[date2] - df[date1]).dt.days
def process_ngaySinh(s):
    if s != s:
        return np.nan
    try:
        s = int(s)
    except ValueError:
        s = s.split(" ")[0]
        
    return datetime.strptime(str(s)[:6], "%Y%m")
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj
def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        if col == "Field_34":
            continue
        df[col] = df[col].apply(date_normalize)

    df["Field_34"] = df["Field_34"].apply(process_ngaySinh)
    
    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        df[col] = df[col].dt.strftime('%d-%m-%Y')
    
    subtract_date('Field_5','Field_6',df)
    subtrac_List = ['Field_1', 'Field_2', 'Field_43', 'Field_44', 'Field_7','Field_8', 'Field_9']
    subtract_2C = list(combinations(subtrac_List, 2))
    for l in subtract_2C:
        subtract_date(l[0],l[1],df)
    
    for cat in ['F', 'E', 'C', 'G', 'A']:
        subtract_date(f'{cat}_startDate', f'{cat}_endDate', df)
    print(df.shape) 
    return df
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s
def combine_gender(s):
    x, y = s
    return x if x != None else y if y != None else np.nan
def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1)
    return df
def process_ordinal(df):        
    df["subscriberCount"].replace(0, np.nan, inplace=True)
    df["friendCount"].replace(0, np.nan, inplace=True)
    
    df["Field_13"] = df["Field_13"].apply(lambda x: 1 if x == x else 0)
    df["Field_38"] = df["Field_38"].map({0: 0.0, 1: 1.0, "DN": np.nan, "TN": np.nan, "GD": np.nan})
    df["Field_62"] = df["Field_62"].map({"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "Ngoài quốc doanh Quận 7": np.nan})
    df["Field_47"] = df["Field_47"].map({"Zezo": 0, "One": 1, "Two": 2, "Three": 3, "Four": 4})
    
    df["Field_27"] = df["Field_27"].replace({0.0: np.nan})
    df["Field_28"] = df["Field_28"].replace({0.0: np.nan})
        
    for col in df.columns:
        if df[col].dtype.name == "object":
            df[col] = df[col].apply(str_normalize).astype("category")
            
    return df
def transform(df):
    df = process_datetime_cols(df)
    df = process_gender(df)
    df = process_ordinal(df)
    return df

In [113]:
df_train = pd.read_csv("../../data/kalapa/train_new.csv")
df_test = pd.read_csv("../../data/kalapa/test_new.csv")
df_all = df_train.drop(['label'], 1).append(df_test)


df_all_fe = transform(df_all.copy())

df_all_fe['Age'] = df_all_fe["ngaySinh 1"].apply(lambda x: 2020 - x)
df_all_fe = df_all_fe.drop('ngaySinh 1', axis = 1)
cols_select = [x for x in df_all_fe.columns if x not in DATE + DATETIME  + [f'{cat}_endDate' for cat in ['F', 'E', 'C', 'G', 'A']] + [f'{cat}_startDate' for cat in ['F', 'E', 'C', 'G', 'A']]]
df_fe = df_all_fe[cols_select]
df_fe.replace([np.inf, -np.inf], -99999, inplace=True)
df_fe["sum_null"] = df_fe.isnull().sum(axis=1)
df_fe.drop(['info_social_sex', 'gioiTinh'], axis = 1)
df_fe.Thue  = df_fe.Thue.apply(lambda x: "no" if x == "nan" else "yes")
y_label = df_train["label"]
train_fe = df_fe[df_fe["id"] < df_train.shape[0]]
test_fe = df_fe[df_fe["id"] >= df_train.shape[0]]


(73411, 203)


/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:



# Label-Encoding
lbl = LabelEncoder()
for col in df_fe.columns:
    if df_fe[col].dtype.name == "category":
        train_fe[col] = train_fe[col].astype(str)
        test_fe[col] = test_fe[col].astype(str)
        encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
        train_fe[col] = train_fe[col].fillna('None')
        test_fe[col] = test_fe[col].fillna('None')

        # Only take the common values in Train/Test set
        common_vals = list(set(train_fe[col]).intersection(set(test_fe[col])))

        # Take if vals appeared both 5 times
        common_vals = set(train_fe[col].value_counts()[train_fe[col].value_counts() > 10].index).intersection(test_fe[col].value_counts()[test_fe[col].value_counts()>4].index)

        # Replace not-common values with "Missing" or np.nan
        train_fe.loc[~train_fe[col].isin(common_vals), col] = 'Missing'
        test_fe.loc[~test_fe[col].isin(common_vals), col] = 'Missing'

        # Implement LE
        lbl.fit(train_fe[col].tolist() + test_fe[col].tolist())
        train_fe[col] = lbl.transform(train_fe[col])
        test_fe[col] = lbl.transform(test_fe[col])

print(train_fe.shape)
print(test_fe.shape)

In [114]:
df_train = pd.concat([df_train.id,df_train.label, train_fe.iloc[:,1:]], axis = 1)

In [115]:
df_train

,id,label,Field_10,Field_19,Field_20,Field_21,Field_23,Field_27,Field_28,Field_29,...,Field_7Field_9,Field_8Field_9,F_startDateF_endDate,E_startDateE_endDate,C_startDateC_endDate,G_startDateG_endDate,A_startDateA_endDate,gender,Age,sum_null
0,0,1,1.0,0.0,4258600.0,4.5,16.0,NaN,NaN,0.0,...,-5.0,-216.0,NaN,NaN,NaN,NaN,67.0,male,29.0,25
1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,nan,NaN,67
2,2,0,1.0,0.0,5000000.0,4.5,10.0,NaN,NaN,0.0,...,16.0,-348.0,0.0,24.0,NaN,NaN,47.0,female,22.0,12
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,71
4,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53025,53025,0,1.0,1.0,1490000.0,4.5,5.0,NaN,NaN,12.0,...,-8.0,-373.0,0.0,0.0,0.0,NaN,202.0,female,32.0,10
53026,53026,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,52.0,nan,NaN,58
53027,53027,0,1.0,0.0,4015000.0,4.5,0.0,NaN,NaN,0.0,...,196.0,-131.0,NaN,NaN,NaN,NaN,NaN,female,32.0,29
53028,53028,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,nan,NaN,67


In [116]:
data_version = "01-02-2021-v4"

In [117]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
df_train.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test_fe.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)